## Run GEODYN for MSIS

In [1]:
run_list = [  'msis2'  ]


In [2]:
# %load_ext autoreload
# %autoreload 2

# import sys  
# import pickle 
# import gc

# sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
# from PYGEODYN import Pygeodyn

# for i,val in enumerate(run_list):   
#     for ii in [ '_p1', '_p2' ]:   

#         run_settings = '/data/zach_work/ML_project_support/runsettings_3month_'+ val +ii+'.yaml'
#         ### Load the data into an object
#         RunObject = Pygeodyn(run_settings)
#         RunObject.RUN_GEODYN()
#         RunObject = 0
#         gc.collect()


## Read GEODYN

In [3]:
# for i,val in enumerate(run_list): 
#     for ii in ['']:
# EDIT_OBJECT_DATES = 0
%load_ext autoreload
%autoreload 2

import pandas as pd
import sys  
import pickle 
import gc
import os

sys.path.insert(0, '/data/geodyn_proj/pygeodyn/pygeodyn_develop/')
from PYGEODYN import Pygeodyn
 
dir_save = '/data/zach_work/output_from_runs/ML_project_support/'


for i,val in enumerate(run_list): 
    for ii in ['_p1', '_p2']:        
        pickle_file = dir_save+'3month_icesat2orbits_'+val+ii+'.pkl'
        if not os.path.exists(pickle_file):
            print('Must create pickle file...')
            print('   ',  pickle_file)
            print('   ', 'Reading Geodyn Data')

            run_settings = '/data/zach_work/ML_project_support/runsettings_3month_'+ val +ii+'.yaml'
            ### Load the data into an object
            Obj_Geodyn = Pygeodyn(run_settings)
            Obj_Geodyn.getData_BigData_lowmemory()
            gc.collect()
            #### Pickle the object to save it
            print('   ', 'Saving pickle')
            filehandler = open(pickle_file, 'wb') 
            pickle.dump(Obj_Geodyn, filehandler)
            filehandler.close()
            Obj_Geodyn = 0
            print('   ', 'Saved pickle')
        else:
            print(val,ii,'  Pickle created -- will load in next step', sep='')
            

### Load the data if the pickles exist
print()
print()
gc.collect()

Obj_Geodyn = {}
for i,val in enumerate(run_list):
    for ii in ['_p1', '_p2']:
        pickle_file = dir_save+'3month_icesat2orbits_'+val+ii+'.pkl'
        filehandler = open(pickle_file, 'rb') 
        Obj_Geodyn[val+ii] = pickle.load(filehandler)
        filehandler.close()
        print('Loaded data... ', '3month_icesat2orbits_',val,ii,'.pkl',sep='')

    


msis2_p1  Pickle created -- will load in next step
msis2_p2  Pickle created -- will load in next step


Loaded data... 3month_icesat2orbits_msis2_p1.pkl
Loaded data... 3month_icesat2orbits_msis2_p2.pkl


In [4]:
Obj_Geodyn['msis2_p1'].__dict__.keys()

# 'Trajectory_orbfil', 'Density', 'Residuals_summary', 'Statistics', 'OrbitResids'

dict_keys(['Trajectory_orbfil', 'Density', 'Residuals_summary', 'Statistics', 'OrbitResids', 'run_parameters2018.292.01', 'global_params', 'run_parameters2018.293.01', 'run_parameters2018.294.01', 'run_parameters2018.295.01', 'run_parameters2018.296.01', 'run_parameters2018.297.01', 'run_parameters2018.298.01', 'run_parameters2018.299.01', 'run_parameters2018.304.01', 'run_parameters2018.305.01', 'run_parameters2018.306.01', 'run_parameters2018.307.01', 'run_parameters2018.308.01', 'run_parameters2018.313.01', 'run_parameters2018.314.01', 'run_parameters2018.315.01', 'run_parameters2018.316.01', 'run_parameters2018.317.01', 'run_parameters2018.318.01', 'run_parameters2018.319.01', 'run_parameters2018.320.01', 'run_parameters2018.321.01', 'run_parameters2018.322.01', 'run_parameters2018.323.01', 'run_parameters2018.324.01', 'run_parameters2018.325.01', 'run_parameters2018.326.01', 'run_parameters2018.327.01', 'arcdate_v2'])

In [5]:
Obj_Geodyn['msis2_p1'].__dict__['OrbitResids']['2018.292.01'].keys()

dict_keys(['data_orbfil', 'data_PCE', 'resids'])

In [6]:
for ii,key2 in enumerate(Obj_Geodyn['msis2_p1'].__dict__['Density']):
    print(key2)
    
    
#     print(Obj_Geodyn['msis2_p1'].__dict__['Density'][key2])
    

2018.292.01
2018.293.01
2018.294.01
2018.295.01
2018.296.01
2018.297.01
2018.298.01
2018.299.01
2018.304.01
2018.305.01
2018.306.01
2018.307.01
2018.308.01
2018.313.01
2018.314.01
2018.315.01
2018.316.01
2018.317.01
2018.318.01
2018.319.01
2018.320.01
2018.321.01
2018.322.01
2018.323.01
2018.324.01
2018.325.01
2018.326.01
2018.327.01


# Save data to CSVs

In [7]:
import os
files_list = [dir_save+'Density_msis2_allarcs.csv',
              dir_save+'Statistics_msis2_allarcs.csv',
              dir_save+'DataOrbfil_msis2_allarcs.csv',
              dir_save+'DataPCE_msis2_allarcs.csv',
              dir_save+'DataResids_msis2_allarcs.csv']
### Delete the existing files so they append cleanly
for file in files_list:
    if os.path.exists(file):
        os.remove(file)

        
append_headers = True      


### prepare the split datasets to be combined
Objs_list = [ Obj_Geodyn['msis2_p1'] , Obj_Geodyn['msis2_p2'] ]

for obj_m1 in Objs_list:

    ### Loop through the arcs and append the datasets for each arc into CSV files
    for ii,arc in enumerate(obj_m1.__dict__['global_params']['arc_input'][:]):
        

        arc =arc+'.01'

        ### ----------------------------------------------------------------------------------
        ### SAVE DENSITY
        ##
        obj_m1.__dict__['Density'][arc].to_csv(path_or_buf = files_list[0],
                                                         sep=',',
                                                         mode='a',
                                                         #
                                                         index_label='Index',
                                                         header=append_headers,
                                                         index=True,
                                                         )
        ### ----------------------------------------------------------------------------------
        ### SAVE STATISTICS
        ### ---------------
        obj_m1.__dict__['Statistics'][arc].to_csv(path_or_buf = files_list[1],
                                                         sep=',',
                                                         mode='a',
                                                         #
                                                         index_label='Index',
                                                         header=append_headers,
                                                         index=True,
                                                         )
        ### ----------------------------------------------------------------------------------
        ### SAVE ORBIT DATASETS 
        ### -------------------   
        ##
        ### Convert the dictionaries to dataframes
        data_orbfil_df = pd.DataFrame.from_dict(obj_m1.__dict__['OrbitResids'][arc]['data_orbfil'], orient='columns')
        data_PCE_df    = pd.DataFrame.from_dict(obj_m1.__dict__['OrbitResids'][arc]['data_PCE'],    orient='columns')
        resids_df      = pd.DataFrame.from_dict(obj_m1.__dict__['OrbitResids'][arc]['resids'],      orient='columns')



        data_orbfil_df.to_csv(path_or_buf = files_list[2],
                                                         sep=',',
                                                         mode='a',
                                                         #
                                                         index_label='Index',
                                                         header=append_headers,
                                                         index=True,
                                                                  )
        data_PCE_df.to_csv(path_or_buf = files_list[3],
                                                         sep=',',
                                                         mode='a',
                                                         #
                                                         index_label='Index',
                                                         header=append_headers,
                                                         index=True,
                                                                  )
        resids_df.to_csv(path_or_buf = files_list[4],
                                                         sep=',',
                                                         mode='a',
                                                         #
                                                         index_label='Index',
                                                         header=append_headers,
                                                         index=True,
                                                                  )
        if ii==0 and append_headers==True:
            append_headers=False
        




# Read data back in from CSVs

In [8]:
files_list = [dir_save+'Density_msis2_allarcs.csv',
              dir_save+'Statistics_msis2_allarcs.csv',
              dir_save+'DataOrbfil_msis2_allarcs.csv',
              dir_save+'DataPCE_msis2_allarcs.csv',
              dir_save+'DataResids_msis2_allarcs.csv']


test_A = pd.read_csv(files_list[3], 
                     sep=',',
                    )



Density_msis2_df = pd.read_csv(files_list[0], 
                     sep=',',
                    )
Statistics_msis2_df = pd.read_csv(files_list[1], 
                     sep=',',
                    )
DataOrbfil_msis2_df = pd.read_csv(files_list[2], 
                     sep=',',
                    )
DataPCE_msis2_df = pd.read_csv(files_list[3], 
                     sep=',',
                    )
DataResids_msis2_df = pd.read_csv(files_list[4], 
                     sep=',',
                    )


In [9]:
DataResids_msis2_df

,Index,Date,N,T,W
0,0,2018-10-19 00:03:00,-0.713981,-11.197472,-0.087724
1,1,2018-10-19 00:05:00,-0.717382,-11.036047,-0.088147
2,2,2018-10-19 00:07:00,-0.715938,-10.874191,-0.086992
3,3,2018-10-19 00:09:00,-0.709690,-10.713108,-0.084288
4,4,2018-10-19 00:11:00,-0.698757,-10.554032,-0.080091
...,...,...,...,...,...
33741,713,2019-01-08 23:49:00,1.559614,-36.156976,-0.013955
33742,714,2019-01-08 23:51:00,1.548662,-36.469961,-0.021459
33743,715,2019-01-08 23:53:00,1.535229,-36.778322,-0.028695
33744,716,2019-01-08 23:55:00,1.519619,-37.081719,-0.035536
